In [ ]:
from matplotlib import pyplot as plt

from nilearn import plotting, datasets
import nibabel as nb

from gsp_neuro import data_loading as dload 
from gsp_neuro import utils as ut
from gsp_neuro import plotting as viz
from gsp_neuro import interactive
from gsp_neuro.brain import Brain

from importlib import reload
reload(dload)
reload(ut)
reload(viz)
reload(interactive)

In [ ]:
data_base_directory = "/Users/hugofluhr/chuv/data/"

In [ ]:
subject_ids = dload.get_subjects(data_base_directory)

In [ ]:
w_sub0 = interactive.select_subject(subject_ids)

In [ ]:
subject = w_sub0.value
print('Current subject is : {}'.format(subject))

In [ ]:
w_scale = interactive.select_scale()

In [ ]:
scale = w_scale.value
print('Current scale is : {}'.format(scale))

In [ ]:
brain = Brain(data_base_directory, subject, scale)

In [ ]:
brain.load_graph()

In [ ]:
brain.add_signal('thickness')
brain.add_signal('pial_lgi')

In [ ]:
#viz.plot_signal_2d(brain, 1)

In [ ]:
consensus = Brain(data_base_directory, 'consensus', scale=2)
consensus.load_graph()

In [ ]:
reload(viz)
viz.plot_signal_3d(consensus, 1, mesh_type='pial_whole', cmap="Spectral", black_bg=False, title='Harmonic')

In [ ]:
reload(viz)
p = viz.plot_connectome(consensus, thresh=96.6, output_file='/Users/hugofluhr/chuv/repositories/gsp_neuro/figures/connectome.svg')

## Arbitrary graph signal

In [ ]:
import numpy as np
from pygsp import graphs

def plot_signal_on_pruned_graph(brain, signal, title=None):

    small_graph = brain.G.subgraph(np.arange(68))
    small_graph = graphs.Graph(
        ut.prune_adjacency(small_graph.W.toarray(), 92),
        coords=small_graph.coords[:, (1, 2)],
    )
    small_graph.plotting["edge_color"] = (0.5, 0.5, 0.5, 1)

    plt.set_cmap("plasma")
    fig_signal, ax = small_graph.plot(
        brain.signals[signal][:68], edges=True, vertex_size=300, title=title
    )
    fig_signal.set_size_inches(8, 4)
    plt.axis("off")

    return fig_signal

In [ ]:
_ = plot_signal_on_pruned_graph(brain, 'thickness')

# Cortical Thickness

In [ ]:
viz.plot_signal_3d(brain, 'pial_lgi', mesh_type='pial_whole', cmap="plasma", black_bg=False, title='LGI', vmin=0, vmax=5)


In [ ]:
sig = brain.get_signal('pial_lgi')

In [ ]:
noise = np.random.normal(0, .5, sig.shape)
noisy_sig = sig + noise

In [ ]:
viz.plot_signal_3d(brain, noisy_sig, mesh_type='pial_whole', cmap="plasma", black_bg=False, title='Noisy LGI', vmin=0, vmax=5)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
viz.plot_spectrum(brain.G, sig, ax=ax)
ax.set_title('Recovered Cortical Thickness')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
viz.plot_spectrum(brain.G, noisy_sig, ax=ax)
ax.set_title('Recovered Cortical Thickness')
plt.show()

In [ ]:
def filter(G, sig, g):
    f_hat = G.U.T@sig
    return G.U@(g*f_hat)


In [ ]:
lp = np.zeros((brain.G.N))
lp[:50]=1.

filtered_sig = filter(brain.G, noisy_sig, lp)

In [ ]:
viz.plot_signal_3d(brain, 'pial_lgi', mesh_type='pial_whole', cmap="plasma", black_bg=False, title='LGI', vmin=0, vmax=5)

In [ ]:
viz.plot_signal_3d(brain, filtered_sig, mesh_type='pial_whole', cmap="plasma", black_bg=False, title='Filtered Noisy LGI', vmin=0, vmax=5)

In [ ]:
viz.plot_signal_3d(brain, noisy_sig, mesh_type='pial_whole', cmap="plasma", black_bg=False, title='Noisy LGI', vmin=0, vmax=5)